In [1]:
#!/usr/bin/env python

import librosa
import midi 
import os
import glob
import matplotlib.pyplot as plt
import numpy as np
import pretty_midi

import config.nn_config as nn_config


In [2]:
nn_params = nn_config.get_nn_params()
drum_keys = { 35:36, # acoustic bass drum -> bass drum (36)
                    37:37, # 37:side stick, 
                    40:38, # 38: acou snare, 40: electric snare
                    43:41, # 41 low floor tom, 43 ghigh floor tom
                    47:45, # 45 low tom, 47 low-mid tom
                    50:48, # 50 high tom, 48 hi mid tom
                    44:42, # 42 closed HH, 44 pedal HH
                    57:49, # 57 Crash 2, 49 Crash 1
                    59:51, # 59 Ride 2,
                    53:51, 
                    55:51, # 51 Ride 1, 53 Ride bell, 55 Splash
                    52:49  # 52: China cymbal
}
# 0 35, 36 - bass drum
# 1 37 - side stick
# 2 38,39,40 - snare
# 3 41 - floor tom
# 4 42 - closed hh
# 5 43 - high floor tom
# 6 
def map2drum(midinote):
    weirdoinstruments = range(60,82)
    if np.isin(midinote, [35,36]):
        note = 1 # bass drum
    if np.isin(midinote,[37]):
        note = 2 # side stick
    if np.isin(midinote, [38,39,40]):
        note = 3 # snares
    if np.isin(midinote, [41, 45, 47]):
        note = 4 # low toms
    if np.isin(midinote, [43, 48, 50]):
        note = 5 # high tom
    if np.isin(midinote, [42, 44, 46]):
        note = 6 # hh 
    if np.isin(midinote, [51, 53 , 54, 56, 59]):
        note = 7 # ride
    if np.isin(midinote, [49, 52, 55, 57, 58]):
        note = 8 # crash
    if np.isin(midinote, weirdoinstruments):
        note = 9 # other
    return note

'onsetsdir' : 'np-data/onsets/metallica/'
'MIDI sequence directory' : 'plots/MIDI/metallica/'
'sequence length' : 100
'sourcedir' : 'audio/metallica/'
'midi dir' : 'midi/metallica/'
'models dir' : 'models/metallica/'
'batchsize' : 10
'ADT sequence directory' : 'plots/ADT/metallica/'
'weightsdir' : 'model-weights/metallica/'
'dev data dir' : 'np-data/dev-data/metallica/'
'model type' : 'lstm'
'epochs' : 3
'rendered midi dir' : 'midi/rendered-metallica/'
'drummer' : 'metallica'
'training data dir' : 'np-data/training-data/metallica/'
'slicedir' : 'audio/sliced/metallica/'
'loss directory' : 'plots/loss/metallica/'
'activationsdir' : 'np-data/activations/metallica/'


In [12]:
nn_params['epochs']

3

In [3]:
# get total tick length from midifile 
def getMidiInfo(midifile):
    
    try:
        pattern = midi.read_midifile(midifile)
        pattern.make_ticks_abs()
        notracks = 0
        for x in range(len(pattern)):
            for y in range(len(pattern[x])):
                if "midi.EndOfTrackEvent" in str(pattern[x][y]):
                    #print 'midi.EndOfTrackEvent found' 
                    endtime = pattern[x][y].tick 
                    #print 'last tick is', endtime
                if "midi.TrackNameEvent" in str(pattern[x][y]):
                    #if pattern[x][0].text in drumkeys:
                    notracks = notracks + 1
            print 'there were %r midi tracks found' %(notracks)
        return int(endtime), notracks  
    
    except Exception as e:
        print 'could not read %r ... skipping' %(midifile) 
        print str(e)
        endtime = 0 
        notracks = 0 
        return int(endtime), notracks
    #return endtime, notracks
    

In [26]:
##### parse and process MIDI Files

def parseMultiMidi3D(midifile, totalticks, seqlen, subdivision=4, num_output_sequences=8):
    #print 'totalticks',totalticks
    #print 'seqlen', seqlen
    tick = [] # empty tick to hold ticks
    notes = [] # empty note list to hold notes 
    uniquenotes = []
    size = totalticks+1
    
    drum_range = range(35,82)
    print "reading", midifile
    
   # quantizetargetlen = quantizelen

    pattern = midi.read_midifile(midifile)
    pattern.make_ticks_abs()
    
    resolution = pattern.resolution
    quant_divisor = resolution/subdivision
    #print 'quantization divisor is', quant_divisor
    #print 'res:', resolution
    p = pretty_midi.PrettyMIDI(midifile)
    totalbeats = int(totalticks/resolution)
    #print 'totalbeats', totalbeats
    quantlen = totalbeats*subdivision
    #print 'quantization len', quantlen
    

    for inst in pattern:
        tick = []
        notes = []
        thesum = 0
        for event in inst:
            if isinstance(event, midi.NoteOnEvent):
                if np.isin(event.data[0], drum_range):
                    if event.data[1] != 0:
                        #md = map2drum(event.pitch)
                        notes.append(map2drum(event.pitch))
                        qtick = event.tick/quant_divisor
                        tick.append(qtick)
                    else:
                        notes.append(0)
                        qtick = event.tick/quant_divisor
                        tick.append(qtick)
            if isinstance(event, midi.NoteOffEvent):
                notes.append(0)
                tick.append(event.tick/quant_divisor)

        ticknote = zip(tick,notes)
        totalticks = ticknote[-1][0]
        print 'total ticks', totalticks

        
        tks = [elem[0] for elem in ticknote] # get ticks (first element) in tuples list
        nts = [elem[1] for elem in ticknote] # get note numbers (2nd elem) in tuples list
        nts = np.array(nts)

#         bins = np.linspace(0, totalticks, quantlen)
#         idx = np.digitize(tks, bins) 
#         idx = np.array(idx)
        
        mat = np.zeros((totalticks, 10))
        
        for t, note in zip(tks, nts):
            if note != 0:
                mat[int(t)-1][note] = 1
            else:
                mat[int(t)-1][note] = 0
    #print 'mat len', mat.shape
    Xt, Yt = sliceData3D(mat, seqlen=seqlen, no_output_sequences=num_output_sequences)
    #Xt, Yt = sliceData3Dim(mat, seqlen=seqlen, no_output_sequences=num_output_sequences)
    #print Xt.shape, Yt.shape
  
    return mat, Xt, Yt


    #except Exception as e:
#         print 'could not process %r  <SKIPPING>' % (midifile)
#         print str(e)
#         bigdaddy = np.array([[0]])
#         qdaddy = np.array([[0]])
#         uniquenotes = [0]
#         return qdaddy.T, uniquenotes
    





In [4]:
##### parse and process MIDI Files

def parseMultiMidi(midifile, totalticks, seqlen, subdivision=4, num_output_sequences=8):
    #print 'totalticks',totalticks
    #print 'seqlen', seqlen
    tick = [] # empty tick to hold ticks
    notes = [] # empty note list to hold notes 
    uniquenotes = []
    size = totalticks+1
    
    drum_range = range(35,82)
    print "reading", midifile
    
   # quantizetargetlen = quantizelen

    pattern = midi.read_midifile(midifile)
    pattern.make_ticks_abs()
    
    resolution = pattern.resolution
    quant_divisor = resolution/subdivision
    #print 'quantization divisor is', quant_divisor
    #print 'res:', resolution
    p = pretty_midi.PrettyMIDI(midifile)
    totalbeats = int(totalticks/resolution)
    #print 'totalbeats', totalbeats
    quantlen = totalbeats*subdivision
    #print 'quantization len', quantlen
    

    for inst in pattern:
        tick = []
        notes = []
        thesum = 0
        for event in inst:
            if isinstance(event, midi.NoteOnEvent):
                if np.isin(event.data[0], drum_range):
                    if event.data[1] != 0:
                        #md = map2drum(event.pitch)
                        notes.append(map2drum(event.pitch))
                        qtick = event.tick/quant_divisor
                        tick.append(qtick)
                    else:
                        notes.append(0)
                        qtick = event.tick/quant_divisor
                        tick.append(qtick)
            if isinstance(event, midi.NoteOffEvent):
                notes.append(0)
                tick.append(event.tick/quant_divisor)

        ticknote = zip(tick,notes)
        totalticks = ticknote[-1][0]
        print 'total ticks', totalticks

        
        tks = [elem[0] for elem in ticknote] # get ticks (first element) in tuples list
        nts = [elem[1] for elem in ticknote] # get note numbers (2nd elem) in tuples list
        nts = np.array(nts)

#         bins = np.linspace(0, totalticks, quantlen)
#         idx = np.digitize(tks, bins) 
#         idx = np.array(idx)
        
        mat = np.zeros((totalticks, 10))
        
        for t, note in zip(tks, nts):
            if note != 0:
                mat[int(t)-1][note] = 1
            else:
                mat[int(t)-1][note] = 0
    #print 'mat len', mat.shape
    Xt, Yt = sliceData(mat, seqlen=seqlen)
    #Xt, Yt = sliceData3Dim(mat, seqlen=seqlen, no_output_sequences=num_output_sequences)
    #print Xt.shape, Yt.shape
  
    return mat, Xt, Yt


    #except Exception as e:
#         print 'could not process %r  <SKIPPING>' % (midifile)
#         print str(e)
#         bigdaddy = np.array([[0]])
#         qdaddy = np.array([[0]])
#         uniquenotes = [0]
#         return qdaddy.T, uniquenotes
    




In [5]:
def parseMidiDir(mididir, seqlen=20):
    #plt.figure()
    #print 'seq len', seqlen
    X_mat = np.empty((0, seqlen , 10))
    Y_mat = np.empty((0, seqlen, 10))
    #print 'parsemidi dir shapes %r %r' %(X_mat.shape, Y_mat.shape)
    cnt = 1
    slen = seqlen
    
    p,d,files = next(os.walk(mididir))
    total_midi_files = str(len(files))
    for midifile in glob.glob(mididir + "/*.mid"):
        print "processing %r/%r  : %r with seqlen: %r" %(str(cnt), total_midi_files, midifile,seqlen)
        endtimes, notracks = getMidiInfo(midifile)
        print endtimes, notracks
        mm, x, y = parseMultiMidi(midifile, endtimes, seqlen=seqlen)
        #print 'after parsemultimidi %r %r' %(x.shape, y.shape)
        X_mat = np.vstack((X_mat, x))
        Y_mat = np.vstack((Y_mat, y))
        #print 'X_mat shape', X_mat.shape
        #print "Y_mat shape", Y_mat.shape
        cnt = cnt+1
        print ' '
        
        #bd_ = np.reshape((1, bd.shape[0], bd.shape[1]))
        ## FORM BIG MATRIX HERE BY STACKING
        
        ######
        print 'size of bigdaddy: %r and unique entries: %r' %(bd.shape, un)
        print ""
        plt.subplot(20,1,cnt)
        for instrument in un:
            plt.stem(bd[instrument-1, :500]*sizearr[cnt])
            plt.show()
        cnt = cnt + 1
    xfile = nn_params["training data dir"] + nn_params["drummer"]  + str(nn_params["sequence length"]) + "_X.npy"
    yfile = nn_params["training data dir"] + nn_params["drummer"]  + str(nn_params["sequence length"]) + "_Y.npy"

    
    np.save(xfile, X_mat)
    np.save(yfile, Y_mat)

    
    return X_mat, Y_mat


        

In [18]:
def parseMidiDir3D(mididir, seqlen=20, num_output_sequences=8):
    #plt.figure()
    #print 'seq len', seqlen
    X_mat = np.empty((0, num_output_sequences, 10))
    Y_mat = np.empty((0, num_output_sequences, 10))
    #print 'parsemidi dir shapes %r %r' %(X_mat.shape, Y_mat.shape)
    cnt = 1
    slen = seqlen
    
    p,d,files = next(os.walk(mididir))
    total_midi_files = str(len(files))
    for midifile in glob.glob(mididir + "/*.mid"):
        print "processing %r/%r  : %r with seqlen: %r" %(str(cnt), total_midi_files, midifile,seqlen)
        endtimes, notracks = getMidiInfo(midifile)
        print endtimes, notracks
        mm, x, y = parseMultiMidi3D(midifile, endtimes, seqlen=seqlen, num_output_sequences=num_output_sequences)
        print 'after parsemultimidi %r %r' %(x.shape, y.shape)
        
        X_mat = np.vstack((X_mat, x))
        Y_mat = np.vstack((Y_mat, y))
        #print 'X_mat shape', X_mat.shape
        #print "Y_mat shape", Y_mat.shape
        cnt = cnt+1
        print ' '
        
        #bd_ = np.reshape((1, bd.shape[0], bd.shape[1]))
        ## FORM BIG MATRIX HERE BY STACKING
        
        ######
#         print 'size of bigdaddy: %r and unique entries: %r' %(bd.shape, un)
#         print ""
#         plt.subplot(20,1,cnt)
#         for instrument in un:
#             plt.stem(bd[instrument-1, :500]*sizearr[cnt])
#             plt.show()
#         cnt = cnt + 1
#     xfile = nn_params["training data dir"] + nn_params["drummer"] + "_seq2seq" + str(nn_params["sequence length"]) + "_X.npy"
#     yfile = nn_params["training data dir"] + nn_params["drummer"] + "_seq2seq" + str(nn_params["sequence length"]) + "_Y.npy"
    xfile = nn_params["training data dir"] + nn_params["drummer"] + "_seq2seq" + "100r" + "_X.npy"
    yfile = nn_params["training data dir"] + nn_params["drummer"] + "_seq2seq" + "100r" + "_Y.npy"

    
    
    np.save(xfile, X_mat)
    np.save(yfile, Y_mat)

    
    return X_mat, Y_mat


        

In [29]:
def sliceData3D(midimat, seqlen=20, no_output_sequences=8):
    print 'midimat shape:', midimat.shape
    X = np.empty((0, seqlen, midimat.shape[1])) 
    Y = np.empty((0, seqlen, midimat.shape[1]))
    #print 'post slicedata3dmin', X.shape


    total_len = midimat.shape[0] # ()
    #print 'total len:', total_len
    
    for i in range(seqlen, total_len, 1):
        #print (i-seqlen), i
        midislice = midimat[(i - seqlen): i, :]
        #print 'midislice.shape', midislice.shape
        x_t = midislice[0:no_output_sequences, :] # the training time steps with last time step sliced off
        y_t = midislice[-no_output_sequences: ] 
        print 'x_t after midislice shape:', x_t.shape
        print 'y_t shape midislice:', y_t.shape
        for len
        #print ''
        # reshape arrays to prepare to stack
        x_t = np.reshape(x_t, (1, x_t.shape[0], x_t.shape[1]))
        y_t = np.reshape(y_t, (1, y_t.shape[0], y_t.shape[1]))
        # vertical stack onto x_tr and y_tr mats
        X = np.vstack((X, x_t))
        Y = np.vstack((Y, y_t))

    return X, Y

In [7]:
nn_params["midi dir"] 

'midi/steve-gadd/'

In [30]:
x, y = parseMidiDir3D('midi/metallica/', seqlen=100, num_output_sequences=100)


processing '1'/'58'  : 'midi/metallica/Lepper Messiah.mid' with seqlen: 100
there were 1 midi tracks found
411360 1
reading midi/metallica/Lepper Messiah.mid
total ticks 3428
midimat shape: (3428, 10)
x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape midi

x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape m

x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape m

x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape m

x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape midislice: (100, 10)
x_t after midislice shape: (100, 10)
y_t shape m

KeyboardInterrupt: 

In [8]:
def sliceData(midimat, seqlen=20):
    #print 'midimat shape:', midimat.shape
    X = np.empty((0, seqlen - 1, midimat.shape[1]))    
    Y = np.empty((0, midimat.shape[1]))
    total_len = midimat.shape[0] # ()
    #print 'total len:', total_len
    
    for i in range(seqlen, total_len, 1):
        #print (i-seqlen), i
        midislice = midimat[(i - seqlen): i, :]
        #print 'midislice.shape', midislice.shape
        x_t = midislice[0:-1, :] # the training time steps with last time step sliced off
        y_t = x_t[-1, :] 
        #print 'x_t shape:', x_t.shape
        #print 'y_t shape:', y_t.shape
        # reshape arrays to prepare to stack
        x_t = np.reshape(x_t, (1, x_t.shape[0], x_t.shape[1]))
        y_t = np.reshape(y_t, (1, y_t.shape[0]))
        # vertical stack onto x_tr and y_tr mats
        X = np.vstack((X, x_t))
        Y = np.vstack((Y, y_t))

    return X, Y

In [ ]:
print tn.shape
plt.figure(figsize=(20,5))
plt.stem(tv[0:200,8])
plt.stem(tv[0:200,7]*0.2)
plt.stem(tv[0:200,6]*0.8)
plt.stem(tv[0:200,3]*0.6)
plt.stem(tv[0:200,1]*0.3)
plt.show()

In [19]:
xtr = np.load('np-data/training-data/metallica/metallica_seq100_X.npy')
ytr = np.load('np-data/training-data/metallica/metallica_seq100_Y.npy')
print xtr.shape, ytr.shape

(168842, 99, 10) (168842, 10)


In [9]:
def quantize(notesarray, targetlen, keepAmplitudes=True):
    # function to quantize sequences to desired length 
    # keepAmplitudes = True # retain original amplitudes 
    len_source = notesarray.shape[0] # 
    posarr = np.nonzero(notesarray)
    #print 'source array shape', notesarray.shape
    #print 'want to quantize to %r length' %(targetlen)
    
    tvect = np.zeros(targetlen) # vector to hold target notes array
    bins = np.linspace(0, len_source - 1, targetlen)
    
    idx = np.digitize(posarr, bins)
    if keepAmplitudes:
        tvect[idx-1] = notesarray[posarr]
    else:
        tvect[idx-1] = 1 
    return tvect

def getUniqueInstruments(bdarray):
    nonzeroentries = np.nonzero(bdarray)[1]
    print nonzeroentries
    thearr = np.unique(nonzeroentries)
    return thearr  

def plotMIDIFile(midimatrix):
    thedrums = getUniqueInstruments(midimatrix)
    print 'the unique midi drums are:', thedrums
    plt.figure(figsize=(20,10))

    coeff = 0.99 # scale each additional drum down so we can see it below prev instr
    
    for inst in thedrums:
        plt.stem(bd[:, inst]*coeff)
        coeff *= coeff 
    plt.show()

In [11]:
nn_params["MIDI sequence directory"]

'plots/MIDI/metallica/'

In [10]:
bd.shape

(200, 127)

In [416]:
fname = "midi-files/" + "100 6-8 Afro 01 F4.mid"
endtimes, notracks = getMidiInfo(fname)
bd, un = parseMulti(fname, endtimes)

midi.EndOfTrackEvent found
last tick is 0
there were 0 midi tracks found
midi.EndOfTrackEvent found
last tick is 1363
there were 0 midi tracks found
reading midi-files/100 6-8 Afro 01 F4.mid


In [12]:
# sanity test for sliceData 
xtr = np.load(nn_params["training data dir"] + nn_params["drummer"] + "_seq" + str(nn_params["sequence length"]) + "_X.npy")
ytr = np.load(nn_params["training data dir"] + nn_params["drummer"] + "_seq" + str(nn_params["sequence length"]) + "_Y.npy")


In [40]:
xtr.shape, ytr.shape

NameError: name 'xtr' is not defined

In [42]:
x.shape, y.shape

((174143, 6, 10), (174143, 10))

In [41]:
np.save("np-midi/training-data/metallica/xtr-seq7.npy", x)
np.save("np-midi/training-data/metallica/ytr-seq7.npy", y)



IOError: [Errno 2] No such file or directory: 'np-midi/training-data/metallica/xtr-seq7.npy'

In [25]:
plt.rcParams['figure.figsize'] = 10,5
bd, un = parseMidiDir("midi/all-midi/jazz")


could not read 'midi/all-midi/jazz/jazz 2.mid' ... skipping
0 0
reading midi/all-midi/jazz/jazz 2.mid
could not process 'midi/all-midi/jazz/jazz 2.mid'  <SKIPPING>
index 24 is out of bounds for axis 0 with size 1
midimat shape: (1, 1)
total len: 1
could not read 'midi/all-midi/jazz/jazz 3.mid' ... skipping
0 0
reading midi/all-midi/jazz/jazz 3.mid
could not process 'midi/all-midi/jazz/jazz 3.mid'  <SKIPPING>
index 24 is out of bounds for axis 0 with size 1
midimat shape: (1, 1)
total len: 1
could not read 'midi/all-midi/jazz/jazz 1.mid' ... skipping
0 0
reading midi/all-midi/jazz/jazz 1.mid
could not process 'midi/all-midi/jazz/jazz 1.mid'  <SKIPPING>
index 24 is out of bounds for axis 0 with size 1
midimat shape: (1, 1)
total len: 1
could not read 'midi/all-midi/jazz/jazz 4.mid' ... skipping
0 0
reading midi/all-midi/jazz/jazz 4.mid
could not process 'midi/all-midi/jazz/jazz 4.mid'  <SKIPPING>
index 24 is out of bounds for axis 0 with size 1
midimat shape: (1, 1)
total len: 1
could no

In [194]:
midifile = "midi-files/mary.mid"
pattern = midi.read_midifile(midifile)
pattern.make_ticks_abs()
for track in pattern:
    #print track
    for event in track:
        if isinstance(event, midi.NoteOnEvent):
            None#print event

In [11]:
##### parse and process MIDI Files
def parseMulti(midifile, totalticks, quantizelen=200):
    
    tick = [] # empty tick to hold ticks
    notes = [] # empty note list to hold notes 
    uniquenotes = []
    size = totalticks+1
    print "reading", midifile
    
    quantizetargetlen = quantizelen

    pattern = midi.read_midifile(midifile)
    resolution = pattern.resolution
    p = pretty_midi.PrettyMIDI(midifile)
    bpm = p.estimate_tempo()
    
    #pattern.make_ticks_abs()

    #headerdata = pattern[0] # get tempo data
    #notedata = pattern[1:] # get noteOn information from midi file 
    #print notedata
    bigdaddy = np.zeros((127, size)) # 46 = total poss num instruments drum set 
    qdaddy = np.zeros((127, quantizetargetlen))
    try:
        for inst in pattern:
            tick = []
            notes = []
            thesum = 0
            for event in inst:
                if isinstance(event, midi.NoteOnEvent):
                    if np.isin(event.data[1], drum_keys):
                        if event.data[1] != 0:

                            notes.append(event.pitch)
                            tick.append(event.tick)
                        else:
                            notes.append(0)
                            tick.append(event.tick)
                if isinstance(event, midi.NoteOffEvent):
                    notes.append(0)
                    tick.append(event.tick)

            ticknote = zip(tick,notes)

            tks = [elem[0] for elem in ticknote] # get ticks (first element) in tuples list
            nts = [elem[1] for elem in ticknote] # get note numbers (2nd elem) in tuples list

            uniqueval = np.unique(nts)
            uniqueval = uniqueval.flatten()
            for elem in uniqueval:
                uniquenotes.append(elem)
            uniquenotes = list(set(uniquenotes))
            uniquenotes.sort()

            #print 'sum of tks', sum(tks)
            sumticks = 0
            tvect = [] # empty list to sum the ticks 
            for i in range(0, len(nts)):        
                sumticks = sumticks + tks[i] # sum the tick with the previous entry
                tvect.append(sumticks)        
            ticknote_ = zip(tvect, nts) # notes in accumulated tick time
            #print ticknote_

            thenotes = [i[1] for i in ticknote_]
            thetimes = [i[0] for i in ticknote_]
            #print thenotes, thetimes

            for time in thetimes:
                inst_idx = [ticknote_[x][1] for x, y in enumerate(ticknote_) if y[0]==time]
                for inst in inst_idx:
                    bigdaddy[inst][time] = 1
                    # QUANTIZE into different time steps 
                    qdaddy[inst] = quantize(bigdaddy[inst, :], quantizetargetlen)
                #print " "

        return qdaddy.T, uniquenotes

    except Exception as e:
        print 'could not process %r  <SKIPPING>' % (midifile)
        print str(e)
        bigdaddy = np.array([[0]])
        qdaddy = np.array([[0]])
        uniquenotes = [0]
        return qdaddy.T, uniquenotes
    



In [151]:
data = np.array([[0.1, 1.0],
[0.2, 0.9],
[0.3, 0.8],
[0.4, 0.7],
[0.5, 0.6],
[0.6, 0.5],
[0.7, 0.4],
[0.8, 0.3],
[0.9, 0.2],
[1.0, 0.1]])

In [152]:
data[:,0]

array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])

In [210]:
bd.shape

(7658, 46)

In [46]:
# print un
# plt.figure()
# cnt = 1
# for inst in un:
#     plt.stem(bd[inst-1, :1000]*(1-0.1*cnt), '--r')
#     cnt = cnt + 1
# plt.show()

In [1]:
# without using make ticks absolute
def parseSingleMidiTrack(midifile, totalticks):
    tick = [] # empty tick to hold ticks
    notes = [] # empty note list to hold notes 
    size = totalticks+1
    print "reading", midifile
    pattern = midi.read_midifile(midifile)
    #pattern.make_ticks_abs()
        
    headerdata = pattern[0] # get tempo data
    notedata = pattern[1:] # get noteOn information from midi file 
    bigdaddy = np.zeros((46, size)) # 46 = total poss num instruments drum set 
    #bigdaddy = [][]
    

    for inst in notedata:
        tick = []
        notes = []
        thesum = 0
        
        notestate = True

        data = inst[1:-1] # slice off midiTrackNameEvent entry and midiEndOfTrack entry
        for note in data:
            try:
                if note.data[1] == 0: # if not noteOff event
#                    print note.tick, note.data[0]
                    notes.append(0) # append value of 0 to note list
                    tick.append(note.tick)
                    #print 'tick: %r  note: 0' %(note.tick)          
                else:
                    if notestate == 
                    notes.append((note.data[0]-34)) # append note to list
                    tick.append(note.tick)
                    #print 'tick: %r  note: %r' %(note.tick, note.data[0]-35)
            except (IOError, IndexError, ValueError):
                pass
            

        ticknote = zip(tick,notes)
        #print 'ticknote', ticknote
        #print ""
    
        tks = [elem[0] for elem in ticknote] # get ticks (first element) in tuples list
        nts = [elem[1] for elem in ticknote] # get note numbers (2nd elem) in tuples list
        
        uniquenotes = np.unique(nts) # get number of unique instruments
        #print 'unique notes', uniquenotes
        
        #print 'sum of tks', sum(tks)
        sumticks = 0
        tvect = [] # empty list to sum the ticks 
        for i in range(0, len(nts)):        
            sumticks = sumticks + tks[i] # sum the tick with the previous entry
            tvect.append(sumticks)        
            #print tks[i], sumticks
        #print 'sum ticks', sumticks
        ticknote_ = zip(tvect, nts) # notes in accumulated tick time
        #print ticknote_
        
        thenotes = [i[1] for i in ticknote_]
        thetimes = [i[0] for i in ticknote_]
        #print 'the times', thetimes
        for time in thetimes:
            inst_idx = [ticknote_[x][1] for x, y in enumerate(ticknote_) if y[0]==time]
            for inst in inst_idx:
                bigdaddy[inst-1][time] = 1
                
        print bigdaddy
        print " "
    
    return bigdaddy, uniquenotes

SyntaxError: invalid syntax (<ipython-input-1-8bf0ef7c245f>, line 32)

In [2]:
# without using make ticks absolute
def parseMultiMidiTrack(midifile, totalticks):
    tick = [] # empty tick to hold ticks
    notes = [] # empty note list to hold notes 
    uniquenotes = []
    size = totalticks+1
    print "reading", midifile
    pattern = midi.read_midifile(midifile)
    #pattern.make_ticks_abs()
        
    headerdata = pattern[0] # get tempo data
    notedata = pattern[1:] # get noteOn information from midi file 

    bigdaddy = np.zeros((46, size)) # 46 = total poss num instruments drum set 
    #biggerdaddy = np.empty((46))
    #bigdaddy = [][]
    

    for inst in notedata:
        #print 'processing %r' %(inst)
        tick = []
        notes = []
        thesum = 0
        data = inst[1:-1] # slice off midiTrackNameEvent entry and midiEndOfTrack entry
        
        thenote = 0 
        notestate = True
        
        for note in data:
            try:
                if note.data[1] == 0: # if not noteOff event
#                    print note.tick, note.data[0]
                    notes.append(0) # append value of 0 to note list
                    tick.append(note.tick)
                    #print 'tick: %r  note: 0' %(note.tick)          
                else:
                    notes.append((note.data[0]-34)) # append note to list
                    tick.append(note.tick)
                    #print 'tick: %r  note: %r' %(note.tick, note.data[0]-35)
            except (IOError, IndexError, ValueError):
                pass
            

        ticknote = zip(tick,notes)
        #print 'ticknote', ticknote
        #print ""
    
        tks = [elem[0] for elem in ticknote] # get ticks (first element) in tuples list
        nts = [elem[1] for elem in ticknote] # get note numbers (2nd elem) in tuples list
        
        uniqueval = np.unique(nts)
        uniqueval = uniqueval.flatten()
        for elem in uniqueval:
            uniquenotes.append(elem)
        uniquenotes = list(set(uniquenotes))
        uniquenotes.sort()

        #print 'sum of tks', sum(tks)
        sumticks = 0
        tvect = [] # empty list to sum the ticks 
        for i in range(0, len(nts)):        
            sumticks = sumticks + tks[i] # sum the tick with the previous entry
            tvect.append(sumticks)        
            #print tks[i], sumticks
        #print 'sum ticks', sumticks
        ticknote_ = zip(tvect, nts) # notes in accumulated tick time
        #print ticknote_
        
        thenotes = [i[1] for i in ticknote_]
        thetimes = [i[0] for i in ticknote_]
        #print 'the times', thetimes
        for time in thetimes:
            inst_idx = [ticknote_[x][1] for x, y in enumerate(ticknote_) if y[0]==time]
            for inst in inst_idx:
                bigdaddy[inst-1][time] = 1
        print bigdaddy
        print " "

        
    return bigdaddy, uniquenotes

In [ ]:
midiroot = './midi-files/'
midifname = midiroot + '100 Displaced 03.mid'
#midifname = '/Users/nolanlem/STANFORD/deep-learning/jazzlstm/cs230/Groove Monkee Free MIDI GM/Jazz Buddy/135 Swing Beat/Swing SN Fill 01.mid'

#totaltks, uniqnotes = getMidiInfo('midi-files/' + midifname)
totaltks, uniqnotes = getMidiInfo(midifname)

print totaltks
#bd, uniqnotes = parseSingleMidiTrack("midi-files/" + midifname, totaltks)
bd, uniqnotes = parseMulti(midifname, totaltks)

In [ ]:
print uniqnotes

In [ ]:
print uniqnotes

plt.figure()
# for nt in uniqnotes:
#     plt.stem(bd[nt-1,:])

plt.stem(bd[2-1, :100])
plt.stem(bd[8-1, :100]*0.3)


plt.show()
    




In [ ]:
tn, tv, bd = parseMidi("midi-files/ecm2.mid")


In [ ]:
print bd.shape

In [ ]:
plt.figure()
plt.stem(bd[0])
plt.show()

In [ ]:
# bd[1][1]
[i for i, e in enumerate(bd[0]) if e!=0]
print bd.shape
plt.figure()
plt.stem(bd[0], 'r')
plt.show()

In [ ]:
[i for i, e in enumerate(bd[0]) if e!=0]
print bd.shape
plt.figure()
plt.stem(bd[0], 'r')
plt.show()

In [ ]:
for x in bd[0]:
    print x

In [8]:
def parseMidi(midifile, totalsize):
    tick = [] # empty tick to hold ticks
    notes = [] # empty note list to hold notes 
    size = totalsize
    print "reading", midifile
    pattern = midi.read_midifile(midifile)
    pattern.make_ticks_abs()
        
    headerdata = pattern[0] # get tempo data
    notedata = pattern[1:] # get noteOn information from midi file 
    bigdaddy = np.empty((0, size))
    #bigdaddy = [][]
    

    for inst in notedata:
        tick = []
        notes = []
        thesum = 0
        data = inst[1:-1] # slice off midiTrackNameEvent entry and midiEndOfTrack entry
        for note in data:
            try:
                if note.data[1] == 0: # if not noteOff event
#                    print note.tick, note.data[0]
                    notes.append(0) # append value of 0 to note list
                    tick.append(note.tick)
                    #print 'tick: %r  note: 0' %(note.tick)          
                else:
                    notes.append((note.data[0]-34)) # append note to list
                    tick.append(note.tick)
                    #print 'tick: %r  note: %r' %(note.tick, note.data[0]-35)
            except (IOError, IndexError, ValueError):
                pass
            

        ticknote = zip(tick,notes)
        #print ticknote
        tvect = np.zeros((1, note.tick))
        #print tvect.shape
        for tk, note in ticknote:
            #print tk, note
            tvect[0][tk] = note
    
    return ticknote, tvect

In [ ]:
print tn
print tv
plt.figure() 
plt.plot(bd[1])
plt.show()

In [ ]:
def parseMidi(mididir):
    tick = [] # empty tick to hold ticks
    notes = [] # empty note list to hold notes 
    for midifile in glob.glob(mididir + "/*.mid"):
        print "reading", midifile
        pattern = midi.read_midifile(midifile)
        headerdata = pattern[0] # get tempo data
        notedata = pattern[1:] # get noteOn information from midi file         
        
        for inst in notedata:
            data = inst[1:-1] # slice off midiTrackNameEvent entry and midiEndOfTrack entry
            for note in data:
                try:
                    if note.data[1] == 0: # if not noteOff event
    #                    print note.tick, note.data[0]
                        notes.append(0) # append tick to list
                    else:
                        notes.append(note.data[0]) # append note to list
                    tick.append(note.tick)
                except (IOError, IndexError, ValueError):
                    pass
            
            ticknote = zip(tick,notes)
            print ticknote
            print ""
    
    nts = [elem[1] for elem in tn]
    sumticks = 0
    tvect = []
    for i in range(0, len(nts)):
        sumticks = sumticks + tks[i] 
        tvect.append(sumticks)
        #print tks[i], sumticks    
    
    return ticknote, tvect

In [ ]:
tn, tv = parseMidi("midi-files")
print len(tn), len(tv)

In [ ]:

tks = [elem[0] for elem in tn]
nts = [elem[1] for elem in tn]

In [ ]:
plt.figure()
plt.stem(tv, tn)
plt.show()

In [ ]:
tks = [elem[0] for elem in tn]
nts = [elem[1] for elem in tn]

In [ ]:
print len(tvect)

In [ ]:
sumticks = 0
tvect = []
for i in range(0, len(nts)):
    sumticks = sumticks + tks[i] 
    tvect.append(sumticks)
    print tks[i], sumticks
   

In [ ]:
print len(nts), len(tvect)

In [ ]:
pattern = midi.read_midifile("midi-files/mary.mid")

In [ ]:
pattern[1]

In [229]:
foo

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.])

In [235]:
tv

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [5]:
drumkeys = {
    "Acoustic Bass Drum" : 35, 
    "Bass Drum 1" : 36, 
    "Side Stick" : 37, 
    "Acoustic Snare" : 38, 
    "Hand Clap" : 39, 
    "Electric Snare" : 40, 
    "Low Floor Tom" : 41, 
    "Closed Hi-Hat" : 42, 
    "High Floor Tom" : 43, 
    "Pedal Hi-Hat" : 44, 
    "Low Tom" : 45, 
    "Open Hi-Hat" : 46, 
    "Low-Mid Tom" : 47, 
    "Hi-Mid Tom" : 48, 
    "Crash Cymbal 1" : 49, 
    "High Tom" : 50, 
    "Ride Cymbal 1" : 51, 
    "Chinese Cymbal" : 52, 
    "Ride Bell" : 53, 
    "Tambourine" : 54, 
    "Splash Cymbal" : 55, 
    "Cowbell" : 56, 
    "Crash Cymbal": 57, 
    "Vibraslap" : 58, 
    "Ride Cymbal 2" : 59, 
    "Hi Bongo" : 60, 
    "Low Bongo" : 61,
    "Mute Hi Conga" : 62, 
    "Open Hi Conga" : 63, 
    "Low Conga" : 64, 
    "High Timbale" : 65, 
    "Low Timbale" : 66, 
    "High Agogo" : 67, 
    "Low Agogo" : 68,
    "Cabasa" : 69,
    "Maracas" : 70, 
    "Short Whistle" : 71, 
    "Long Whistle" : 72,
    "Short Guiro" : 73, 
    "Long Guiro" : 74, 
    "Claves" : 75, 
    "High Wood Block": 76, 
    "Low Wood Block" : 77, 
    "Mute Cuica" : 78, 
    "Open Cuica" : 79, 
    "Mute Triangle" : 80, 
    "Open Triangle" : 81
}